In [ ]:
import torch
import os
from torch import nn
import torchvision
from torchvision import transforms
import  torchvision.models as models
from torchvision.transforms import ToTensor,Compose,Normalize,Resize,CenterCrop,ColorJitter
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from PIL import Image
from torch import nn
import pathlib
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchsummary import summary

In [ ]:
# Importing the dataset

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")
print("Path to dataset files:", path)

In [ ]:
dataroot = path         #directory for the dataset
N_EPOCHS = 100          #No. of epochs to train
BATCH_SIZE = 128        #batch_size
z_dim = 100             #Latent dimensions (dimension of the random noise to be fed to the generator)
N_critic = 1            #No. of times for the critic to be trained per generator iteration
Img_channels = 3            
Input_Shape = [3,128,128]
Hidden_dims = 64        #No. of Hidden channels 
lr = 1e-4               #learning rate
betas = (0.5,0.999)     #beta values for adam optmizer
device = torch.device("cuda:0" if (torch.cuda.is_available() ) else "cpu")




In [ ]:
def weights_init(m):
    
    """ Initializes the parameters of the model"""
    
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Discriminator(nn.Module):
   
    """The critic network for the DC Gan"""

    def __init__(self, Input_channels = Img_channels):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(nn.Conv2d(in_channels=Input_channels, out_channels = Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),    #[3, 128, 128] --> [Hidden_dims, 64, 64]
        nn.LeakyReLU(0.2, inplace=True),
        nn.BatchNorm2d(Hidden_dims),

        nn.Conv2d(in_channels = Hidden_dims, out_channels = Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),   #[Hidden_dims, 64, 64] --> [Hidden_dims, 32, 32]   
        nn.LeakyReLU(0.2, inplace = True),
        nn.BatchNorm2d(Hidden_dims),

        nn.Conv2d(in_channels = Hidden_dims, out_channels = 2*Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),     #[Hidden_dims, 32, 32] --> [2*Hidden_dims, 16, 16]   
        nn.LeakyReLU(0.2, inplace = True),
        nn.BatchNorm2d(2*Hidden_dims),

        nn.Conv2d(in_channels = 2*Hidden_dims, out_channels = 4*Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),       #[2*Hidden_dims, 16, 16] --> [4*Hidden_dims, 8, 8]   
        nn.LeakyReLU(0.2, inplace = True),
        nn.BatchNorm2d(4*Hidden_dims),

        nn.Conv2d(in_channels = 4*Hidden_dims, out_channels = 8*Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),       #[4*Hidden_dims, 16, 16] --> [8*Hidden_dims, 4, 4]  
        nn.LeakyReLU(0.2, inplace = True),
        nn.BatchNorm2d(8*Hidden_dims),
        
        nn.Conv2d(in_channels = 8*Hidden_dims, out_channels = 1, kernel_size = 4, stride = 1, padding = 0, bias = False),       #[8*Hidden_dims, 4, 4] --> [1, 1, 1]  
        nn.Sigmoid())

    def forward(self,x):
        return self.main(x)
        





        

        

In [ ]:
class Generator(nn.Module):

    """ Generator Network for DC GAN with Transpose Convulutional layers for upsampling"""

    def __init__(self, z = z_dim, Output_channels = Img_channels):
        super(Generator, self).__init__()
        
        self.main = nn.Sequential(nn.ConvTranspose2d(in_channels = z_dim, out_channels = 8*Hidden_dims, kernel_size = 4, stride = 1, padding = 0, bias = False),    #[z_dim, 1, 1] --> [8*Hidden_dims, 4, 4]
        nn.BatchNorm2d(8*Hidden_dims),
        nn.LeakyReLU(0.2,True),
        
        nn.ConvTranspose2d(in_channels = 8*Hidden_dims, out_channels = 4*Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),      #[8*Hidden_dims, 4, 4] --> [4*Hidden_dims, 8, 8]
        nn.BatchNorm2d(4*Hidden_dims),
        nn.LeakyReLU(0.2, True),
        
        nn.ConvTranspose2d(in_channels = 4*Hidden_dims, out_channels = 2*Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),      #[4*Hidden_dims, 8, 8] --> [2*Hidden_dims, 16, 16]
        nn.BatchNorm2d(2*Hidden_dims),
        nn.LeakyReLU(0.2, True),
        
        nn.ConvTranspose2d(in_channels = 2*Hidden_dims, out_channels = Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),        #[2*Hidden_dims, 16, 16] --> [Hidden_dims, 32, 32]
        nn.BatchNorm2d(Hidden_dims),
        nn.LeakyReLU(0.2, True),

        nn.ConvTranspose2d(in_channels = Hidden_dims, out_channels = Hidden_dims, kernel_size = 4, stride = 2, padding = 1, bias = False),      #[Hidden_dims, 32, 32] --> [Hidden_dims, 64, 64]
        nn.BatchNorm2d(Hidden_dims),
        nn.LeakyReLU(0.2, True),
        
        nn.ConvTranspose2d(in_channels = Hidden_dims, out_channels = Output_channels, kernel_size = 4, stride = 2, padding = 1, bias = False),      #[Hidden_dims, 64, 64] --> [3, 128, 128]
        nn.Tanh())
    
    def forward(self,x):
        return self.main(x)



In [ ]:
class Generator2(nn.Module):

    """ Generator Network for DC GAN with Upsampling(Nearest2d)+ Convulutional layers  for upsampling"""

    def __init__(self, z = z_dim, Output_channels = Img_channels):
        super(Generator2, self).__init__()
        
        self.main = nn.Sequential(nn.ConvTranspose2d(in_channels = z_dim, out_channels = 4*Hidden_dims, kernel_size = 7, stride = 1, padding = 1, bias = False),        #[z_dims, 1, 1] --> [4*Hidden_dims, 5, 5]        
        nn.LayerNorm([4*Hidden_dims,5,5]),
        nn.LeakyReLU(0.2,True),

        nn.UpsamplingNearest2d(scale_factor=2),     #[4*Hiden_dims, 5, 5] --> [4*Hidden_dims, 10, 10]
        nn.Conv2d(in_channels=4*Hidden_dims,out_channels=2*Hidden_dims,kernel_size=7,stride=1,padding=2),       #[4*Hiden_dims, 10, 10] --> [2*Hidden_dims, 8, 8]
        nn.LayerNorm([2*Hidden_dims,8,8]),
        nn.LeakyReLU(0.2,True),
        
        nn.UpsamplingNearest2d(scale_factor=2),     #[2*Hidden_dims, 8, 8] --> [2*Hidden_dims, 16, 16]
        nn.Conv2d(in_channels=2*Hidden_dims,out_channels=Hidden_dims,kernel_size=5,stride=1,padding=2),     #[2*Hidden_dims, 16, 16] --> [Hidden_dims, 16, 16]
        nn.LayerNorm([Hidden_dims,16,16]),
        nn.LeakyReLU(0.2,True),

        nn.UpsamplingNearest2d(scale_factor=2),     #[Hidden_dims, 16, 16] --> [Hidden_dims, 32, 32]
        nn.Conv2d(in_channels=Hidden_dims,out_channels=Hidden_dims,kernel_size=5,stride=1,padding=2),       #[Hidden_dims, 32, 32] --> [Hidden_dims, 32, 32]
        nn.LayerNorm([Hidden_dims,32,32]),
        nn.LeakyReLU(0.2,True),

        nn.UpsamplingNearest2d(scale_factor=2),     #[Hidden_dims, 32, 32] --> [Hidden_dims, 64, 64]
        nn.Conv2d(in_channels=Hidden_dims,out_channels=Hidden_dims,kernel_size=3,stride=1,padding=1),       #[Hidden_dims, 64, 64] --> [Hidden_dims, 64, 64]
        nn.LayerNorm([Hidden_dims,64,64]),
        nn.LeakyReLU(0.5,True),
        
        nn.UpsamplingNearest2d(scale_factor=2),    #[Hidden_dims, 64, 64] --> [Hidden_dims, 128, 128]
        nn.Conv2d(in_channels=Hidden_dims,out_channels=Output_channels,kernel_size=3,stride=1,padding=1),       #[Hidden_dims, 128, 128] --> [3, 128, 128]
        nn.Tanh())
    
    def forward(self,x):
        return self.main(x)



In [ ]:
# Initializing the Generator and critic network and initialing their weights.
generator = Generator2().to(device)
discriminator = Discriminator().to(device)
weights_init(generator)
weights_init(discriminator)


In [ ]:
summary(discriminator,input_size=(3,128,128))

In [ ]:
summary(generator,input_size=(100,1,1))

In [ ]:
def sample_noise(size = z_dim, batch_size = BATCH_SIZE):

    """Samples latent variable/Noise of size [Batch_size, z_dim, 1, 1] from a normal distribution"""

    return torch.randn(batch_size, z_dim, 1, 1)

In [ ]:
#optimizer for the generator
opt_gen = torch.optim.Adam(generator.parameters(),lr = 0.001,betas=betas)    
#optimizer for the discriminator
opt_disc = torch.optim.Adam(discriminator.parameters(),lr = 0.0001,betas = betas)       


In [ ]:
## Differentiable Augmentations
## Taken from https://github.com/mit-han-lab/data-efficient-gans/blob/master/DiffAugment_pytorch.py
def DiffAugment(x, policy ='color,translation,cutout', channels_first = True):
    if policy:
        if not channels_first:
            x = x.permute(0, 3, 1, 2)
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if not channels_first:
            x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
    return x


def rand_brightness(x):
    x = x + (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device = x.device) - 0.5)
    return x


def rand_saturation(x):
    x_mean = x.mean(dim = 1, keepdim = True)
    x = (x - x_mean) * (torch.rand(x. size(0), 1, 1, 1,
                                   dtype = x.dtype, device = x.device) * 2) + x_mean
    return x


def rand_contrast(x):
    x_mean = x.mean(dim=[1, 2, 3], keepdim = True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1,
                                   dtype=x.dtype, device = x.device) + 0.5) + x_mean
    return x


def rand_translation(x, ratio = 0.125):
    shift_x, shift_y = int(x.size(2) * ratio +
                           0.5), int(x.size(3) * ratio + 0.5)
    translation_x = torch.randint(-shift_x, shift_x + 1,
                                  size=[x.size(0), 1, 1], device = x.device)
    translation_y = torch.randint(-shift_y, shift_y + 1,
                                  size=[x.size(0), 1, 1], device = x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device = x.device),
        torch.arange(x.size(2), dtype=torch.long, device = x.device),
        torch.arange(x.size(3), dtype=torch.long, device = x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = torch.nn.functional.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[
        grid_batch, grid_x, grid_y].permute(0, 3, 1, 2).contiguous()
    return x


def rand_cutout(x, ratio = 0.5):
    cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    offset_x = torch.randint(0, x.size(
        2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
    offset_y = torch.randint(0, x.size(
        3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype = torch.long, device = x.device),
        torch.arange(cutout_size[0], dtype = torch.long, device = x.device),
        torch.arange(cutout_size[1], dtype = torch.long, device = x.device),
    )
    grid_x = torch.clamp(grid_x + offset_x -
                         cutout_size[0] // 2, min=0, max = x.size(2) - 1)
    grid_y = torch.clamp(grid_y + offset_y -
                         cutout_size[1] // 2, min=0, max = x.size(3) - 1)
    mask = torch.ones(x.size(0), x.size(2), x.size(3),
                      dtype = x.dtype, device = x.device)
    mask[grid_batch, grid_x, grid_y] = 0
    x = x * mask.unsqueeze(1)
    return x


AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
}

In [ ]:
#transformations to be applied to the input images such as resizing to 128 x 128 and converting them to tensors.
Transforms = transforms.Compose([transforms.ToTensor(),transforms.Resize((128,128)),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])    
img_dataset = torchvision.datasets.ImageFolder(root  =dataroot,transform = Transforms)      
dataloader = DataLoader(img_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
def train_DC_gan(
        Device = device,
        gen_net = generator,
        disc_net = discriminator,
        Dataloader = dataloader,
        opt_gen = opt_gen,
        opt_disc = opt_disc,
        noise_sampler = sample_noise,
        D_losses = [],
        G_losses = [],
        EPOCHS = N_EPOCHS,
        N_critic = N_critic):
    
    step = 0
    EPS = 1e-9  # for Numerical stability

    for epoch in range(EPOCHS):

        for (X,y) in tqdm(Dataloader):
            X = X.to(device)
            curr_batch_size = X.shape[0]

            #training discriminator
            for _ in range(N_critic):
                #real images
                opt_disc.zero_grad()
                d_real = discriminator(DiffAugment(X))
                loss_d_real = -1*torch.log(d_real+EPS).mean()
                loss_d_real.backward()
                #fake images
                noise = noise_sampler(batch_size=curr_batch_size)
                fake = generator(noise)
                d_fake = discriminator(DiffAugment(fake).detach())
                loss_d_fake = -1*torch.log(1-d_fake+EPS).mean()
                loss_d_fake.backward(retain_graph=True) 
                opt_disc.step()
                
            #training generator
            opt_gen.zero_grad()
            g_fake = discriminator(DiffAugment(fake))
            loss_g = (-1*torch.log(g_fake+EPS)).mean()
            loss_g.backward(retain_graph=True)
            opt_gen.step()
            
            if step %50 == 0:
                print(f"epoch:{epoch+1} iter{step} disc_loss:{(loss_d_fake+loss_d_real)} gen_loss:{loss_g}")
        
            if step % 200 == 0:
                with torch.no_grad():
                # Use the first image from fake_images instead of generating new ones
                    fake_images = fake[0].unsqueeze(0).cpu()  # Take the first image and add batch dimension
                    # Ensure the image is in the range [0, 1]
                    fake = (fake + 1) / 2.0  # Transform from [-1, 1] to [0, 1]
                    img = torchvision.utils.make_grid(fake, normalize=False)
                    img_np = img.detach().permute(1, 2, 0).numpy()  # Add detach() here
                    plt.figure(figsize=(8, 8))
                    plt.imshow(img_np)
                    plt.axis('off')
                    plt.title(f"Epoch {epoch+1}, iter {step}")
                    save_dir = "generated_images"
                    os.makedirs(save_dir,exist_ok= True)
                    save_path = os.path.join(save_dir, f'generated_image_epoch_{epoch+1}_batch_{step+1}.png')
                    plt.savefig(save_path)
                    plt.close()
            step +=1
                
        G_losses.append(loss_g.item())
        D_losses.append(loss_d_fake.item()+loss_d_real.item())
    return G_losses,D_losses
            

In [ ]:
##plotting loss curves
generator_loss,discriminator_loss = train_DC_gan(N_critic = 1)
plt.plot(generator_loss)
plt.plot(discriminator_loss)
plt.xlabel("Number of Epochs")
plt.ylabel("loss")
plt.legend(["generator","discriminator"])
plt.title(f"N_critic = {N_critic}")
plt.show()

In [ ]:
def plot_images(save_path,generator = generator, sample_noise = sample_noise, N_samples = 100,title = None):

    """Plot a grid a fake images generated by the generator"""

    z = sample_noise(batch_size = N_samples)      # sample random noise
    images = generator(z).detach().cpu()          # generate fake images 
    img = torchvision.utils.make_grid(images, nrow = 10, normalize = False)
    img = img.permute(1, 2, 0).numpy()            # convert tensors to numpy array
    img = (img +1)/2.0                            # Transform from [-1, 1] to [0, 1]
    plt.figure(figsize = (10,10))
    plt.axis('off')
    if title:
        plt.title(title)
    plt.imshow(img)
    plt.savefig(save_path)
    

                                ##Frechet Inception Distance       

In [ ]:
from torchvision.models import inception_v3

preprocess = transforms.Compose([       # Define preprocessing steps for Inception v3 input
    transforms.Resize((299, 299)),  # Resize images to Inception v3 input size
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats ie we will also preprocess the data in same way as the original imagenet dataset that was used to train Inception v3
])  

def prepare_inception_input(images):

    """Apply preprocessing to the input images"""

    return preprocess(images)  

In [ ]:
def load_inception_model():

    """Load inception pretrained v3 model for extracting features"""
    
    model = inception_v3(pretrained=True, transform_input=False)  # Load pre-trained Inception v3 model without input transformation
    model.fc = torch.nn.Identity()  # Removes the last fully connected layer
    model.eval()  # Set the model to evaluation mode
    return model.to(device)  # Move the model to the same device as our GAN

inception_model = load_inception_model()  # Load and prepare the modified Inception v3 model

In [ ]:
def extract_features(images):

    """returns features of size [2048] extracted from the inception model"""
    
    with torch.no_grad():
        features = inception_model(images)
    return features  # This will be a tensor of shape [batch_size, 2048]

In [ ]:
def extract_real_features(dataloader, num_images=1000):

    """return features of real images extracted from the inception model"""

    real_features = []  # List to store features of real images
    image_count = 0  # Counter for processed images

    with torch.no_grad():  # Disable gradient computation for efficiency
        for batch in dataloader:
            images = batch[0].to(device)  # Move batch of images to the device (assuming batch[0] contains images)
            batch_size = images.size(0)  # Get the current batch size
            
            if image_count + batch_size > num_images:
                # If adding this batch would exceed num_images, only take what's needed
                images = images[:num_images - image_count]
            
            preprocessed_images = prepare_inception_input(images)  # Preprocess images for Inception v3
            batch_features = extract_features(preprocessed_images)  # Extract features using Inception v3
            
            real_features.append(batch_features)  # Add batch features to the list
            
            image_count += batch_features.size(0)  # Update the count of processed images
            if image_count >= num_images:
                break  # Stop if we've processed enough images

    real_features_tensor = torch.cat(real_features, dim=0)  # Concatenate all features into a single tensor
    return real_features_tensor[:num_images]  # Return exactly num_images features

# Extract features from 1000 real images
real_features = extract_real_features(dataloader, num_images=9000)  # Extract features from 1000 real images

In [ ]:
def generate_and_extract_features(generator, num_samples=1000, batch_size=64):

    """return features of fake images extracted from the inception model"""

    generated_features = []  # List to store features of generated images
    num_batches = (num_samples + batch_size - 1) // batch_size  # Calculate number of batches needed

    generator.eval()  # Set generator to evaluation mode
    print(f"Starting feature extraction for {num_samples} samples with batch size {batch_size}")  # Print start of extraction process
    with torch.no_grad():  # Disable gradient computation for efficiency
        for batch_idx in range(num_batches):
            current_batch_size = min(batch_size, num_samples - len(generated_features))  # Adjust batch size for last batch if needed
            print(f"Processing batch {batch_idx + 1}/{num_batches} with size {current_batch_size}")  # Print current batch information
            noise = torch.randn(current_batch_size, 100, 1, 1, device=device)  # Generate noise for input to generator
            fake_images = generator(noise)  # Generate fake images
            
            preprocessed_images = prepare_inception_input(fake_images)  # Preprocess images for Inception v3
            batch_features = extract_features(preprocessed_images)  # Extract features using Inception v3
            
            generated_features.append(batch_features)  # Add batch features to the list
            print(f"Extracted features shape: {batch_features.shape}")  # Print shape of extracted features
            
            if len(generated_features) * batch_size >= num_samples:
                print(f"Reached target number of samples. Stopping extraction.")  # Print when target samples reached
                break  # Stop if we've generated enough samples
    print(f"Feature extraction completed. Total features extracted: {len(generated_features) * batch_size}")  # Print completion of extraction process

    generated_features_tensor = torch.cat(generated_features, dim=0)  # Concatenate all features into a single tensor
    return generated_features_tensor[:num_samples]  # Return exactly num_samples features

# Generate 1000 samples and extract their features
generated_features = generate_and_extract_features(generator, num_samples=9000)  # Generate and extract features from 1000 fake images

In [ ]:
# Calculate mean and covariance of real features
real_mean = torch.mean(real_features, dim=0)  # Calculate mean across all samples for each feature
real_cov = torch.cov(real_features.T)  # Calculate covariance matrix of features


In [ ]:
# Calculate mean and covariance of generated features
generated_mean = torch.mean(generated_features, dim=0)  # Calculate mean across all samples for each feature
generated_cov = torch.cov(generated_features.T)  # Calculate covariance matrix of features

In [ ]:
def calculate_frechet_inception_distance(real_mean, real_cov, generated_mean, generated_cov):
    """
    Calculate the Fréchet Inception Distance (FID) between real and generated image features.
    
    Args:
    real_mean (torch.Tensor): Mean of real image features.
    real_cov (torch.Tensor): Covariance matrix of real image features.
    generated_mean (torch.Tensor): Mean of generated image features.
    generated_cov (torch.Tensor): Covariance matrix of generated image features.
    
    Returns:
    float: The calculated FID score.
    """
    
    # Convert to numpy for scipy operations
    real_mean_np = real_mean.cpu().numpy()  # Convert real mean to numpy array
    real_cov_np = real_cov.cpu().numpy()  # Convert real covariance to numpy array
    generated_mean_np = generated_mean.cpu().numpy()  # Convert generated mean to numpy array
    generated_cov_np = generated_cov.cpu().numpy()  # Convert generated covariance to numpy array
    
    # Calculate squared L2 norm between means
    mean_diff = np.sum((real_mean_np - generated_mean_np) ** 2)  # Compute squared difference between means
    
    # Calculate sqrt of product of covariances
    covmean = scipy.linalg.sqrtm(real_cov_np.dot(generated_cov_np))  # Compute matrix square root
    
    # Check and correct imaginary parts if necessary
    if np.iscomplexobj(covmean):
        covmean = covmean.real  # Take only the real part if result is complex
    
    # Calculate trace term
    trace_term = np.trace(real_cov_np + generated_cov_np - 2 * covmean)  # Compute trace of the difference
    
    # Compute FID
    fid = mean_diff + trace_term  # Sum up mean difference and trace term
    
    return fid  # Return FID as a Python float



In [ ]:
# Calculate FID for animal dataset using the above function
fid_score = calculate_frechet_inception_distance(real_mean, real_cov, generated_mean, generated_cov)  # Compute FID score
print(f"Fréchet Inception Distance (FID): {fid_score:.4f}")  # Print the calculated FID score